In [1]:
from openai import OpenAI
import os
import json
import time

from secret import API_KEY
client = OpenAI(api_key=API_KEY)

In [2]:
def call_api_summarize(article, mname:str = 'gpt-3.5-turbo', eval_max_tokens:int = 128):
    prompt = f"Summarize the following article within 3 sentences: \n\n{article}"
    msg = [
        {"role": "user", "content": prompt}
    ]
    try:
        response = client.chat.completions.create(
            model = mname,
            messages=msg,
            temperature=0,
            max_tokens=eval_max_tokens
        )
    except Exception as e:
        print("openai experiencing high volume, wait 10s to retry for 1st time...")
        time.sleep(10)
        try:
            response = client.chat.completions.create(
                model = mname,
                messages=msg,
                temperature=0,
                max_tokens=eval_max_tokens
            )
        except Exception as e:
            print("openai experiencing high volume, wait 20s to retry for 2nd time...")
            time.sleep(20)
            response = client.chat.completions.create(
                model = mname,
                messages=msg,
                temperature=0,
                max_tokens=eval_max_tokens
            )
    model_resp = response.choices[0].message.content
    prompt_len = response.usage.prompt_tokens
    total_len = response.usage.total_tokens
    print(model_resp)
    return (model_resp, prompt_len, total_len)
    

In [3]:
# constant
fdir = 'data'
model_name = 'gpt-3.5-turbo'
max_tokens = 128


In [4]:
#build the json to store article and gpt-generated summary
def build_json(id:int, article:str, reference:str, prompt_len:int, total_len:int):
    data = {
        "id": id,
        "article": article,
        "reference": reference,
        "prompt_len": prompt_len,
        "total_len": total_len
    }
    return data

In [5]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [19]:
for i in range(3):
    article = dataset['train'][i]['article']
    reference, prompt_len, total_len = call_api_summarize(article, model_name, max_tokens)
    id = dataset['train'][i]['id']
    obj = build_json(id=id, article=article, reference=reference, prompt_len=prompt_len, total_len=total_len)
    with open(f'{fdir}/cnndm_test.jsonl', 'a') as f:
        f.write(json.dumps(obj))
        f.write('\n')

Daniel Radcliffe gains access to a £20 million fortune as he turns 18, but he insists he won't be extravagant with his money. Despite his growing fame and riches, Radcliffe plans to keep his feet firmly on the ground and avoid going off the rails. He has upcoming projects beyond the Harry Potter series, including a TV movie and an Australian film.
CNN correspondent Soledad O'Brien takes readers inside a Miami jail where mentally ill inmates are housed on the "forgotten floor" before trial, often facing avoidable felonies. Judge Steven Leifman advocates for change, aiming to move mentally ill inmates to a new mental health facility for long-term treatment instead of cycling them through the jail system repeatedly. Leifman believes this solution is a win-win, benefiting the patients, their families, and saving the state money.
Drivers who were on the Minneapolis bridge when it collapsed described harrowing tales of survival, with one survivor helping to rescue children from a school bus 

In [7]:
with open(f'{fdir}/cnndm_test.json', 'r') as f:
    data=json.load(f)

In [11]:
data['reference']

'An empty rocket launcher tube was found on the front lawn of a Jersey City home, prompting an investigation by police and FBI agents. The launcher, a 20-year-old AT4 anti-tank rocket launcher tube, was turned over to U.S. Army officials and deemed not a hazard to public safety. The origin of the launcher and the circumstances surrounding its appearance on residential property are being investigated by the Joint Terrorism Task Force division of the FBI and Jersey City police.'